<a href="https://colab.research.google.com/github/Riccardolotorto/pythonGEOPANDASnuovo/blob/main/Geopandas_list_dict_zip_merge_cascaded_union_matplotlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install geopandas contextily

In [3]:
import pandas as pd
import geopandas as gpd
import contextily as ctx

In [4]:
comuni = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/Com01012022_g")
regioni = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/Reg01012022_g (1)")
province = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/ProvCM01012022_g")
ripartizioni_geografiche = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/Limiti01012022_g/RipGeo01012022_g")

1. una lista contenente l'elenco delle regioni

In [ ]:
listaRegioni = list(set(regioni["DEN_REG"]))
listaRegioni.sort()
listaRegioni

2. un dizionario con l'elenco delle ripartizioni e il numero di regioni in ogni ripartizione

In [ ]:
regioni_ripartizioni = regioni.groupby("COD_RIP")[["DEN_REG"]].count().reset_index()
regioni_ripartizioni = regioni_ripartizioni.merge(ripartizioni_geografiche, on = "COD_RIP")
dizionario = dict(zip(regioni_ripartizioni["DEN_RIP"], regioni_ripartizioni["DEN_REG"]))
dizionario

3. data una regione, la mappa con i confini della regione

In [ ]:
regione = input("inserisci regioni ")
regione_scelta = regioni[regioni["DEN_REG"] == regione.capitalize()].to_crs(3857)
ax = regione_scelta.plot(figsize = (12, 8), edgecolor = "k", facecolor = "none")
ctx.add_basemap(ax)

4.  data una regione, la mappa con i confini della regione in nero e i confini delle province di quella regioni in rosso

In [6]:
province3857 = province.to_crs(3857)

In [ ]:
province_regioni = province3857[province3857.within(regione_scelta.geometry.item())]
ax = province_regioni.plot(figsize = (12, 8), edgecolor = "red", facecolor = "none")
regione_scelta.plot(ax = ax, edgecolor = "k", facecolor = "none")
ctx.add_basemap(ax)

5. data una regione, la mappa con i confini della regione in nero, i confini delle province di quella regioni in rosso e i confini dell'Italia in blu

In [ ]:
regioni3857 = regioni.to_crs(3857)

In [10]:
rip3857 = ripartizioni_geografiche.to_crs(3857)

In [ ]:
from shapely.ops import cascaded_union
itaOggetto = cascaded_union(rip3857.geometry)
italia = gpd.GeoSeries(itaOggetto)

In [ ]:
ax = province_regioni.plot(figsize = (12, 8), edgecolor = "red", facecolor = "none")
regione_scelta.plot(ax = ax, edgecolor = "k", facecolor = "none")
italia.plot(ax = ax, facecolor = "none", edgecolor = "blue", linewidth = 2)
ctx.add_basemap(ax)

6. data una regione, i nomi delle regioni che confinano con quella regione. Visualizzare anche il risultato su una mappa

In [ ]:
regioniConfinanti = regioni3857[regioni3857.touches(regione_scelta.geometry.item())]
regioniConfinanti[["DEN_REG"]]

In [ ]:
ax = regioniConfinanti.plot(figsize = (12, 8), edgecolor = "k", facecolor = "none")  
ctx.add_basemap(ax)

7. l'elenco delle regioni con la loro estensione in kmq. Ordinare in ordine decrescente sull'estensione

In [ ]:
regioni3857["Shape_Area"] = regioni3857["Shape_Area"] / 1000000
regioni3857[["DEN_REG", "Shape_Area"]].sort_values(by = "Shape_Area", ascending = False)

8.  rappresentare i dati dell'esercizio precedente con un grafico a barre orizzontali, a barre verticali e con un grafico a torta. Prestare attenzione alla leggibilità delle informazioni riportate sugli assi

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
labels = regioni3857["DEN_REG"]
dati = regioni3857["Shape_Area"]

fig, ax = plt.subplots(figsize=(15,11))
ax.bar(labels, dati, label='estensione per ogni regione italiana')
plt.xticks(rotation = 80)
ax.set_ylabel('estensione (kmQ)')
ax.set_title('regioni')
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(15,11))
ax.barh(labels, dati, label='estensione per ogni regione italiana')
ax.set_ylabel('regione')
ax.set_title('estensione (kmQ)')
ax.legend()

In [ ]:
plt.figure(figsize=(16, 8))
plt.pie(dati, labels=labels, autopct='%1.1f%%')
plt.show()

9. data una regione, i nomi delle regioni che non confinano con quella regione. Visualizzare anche il risultato su una mappa

In [ ]:
regioniNONConfinanti = regioni3857[~regioni3857.touches(regione_scelta.geometry.item())] #l'ondina rappresenta la negazione
regioniNONConfinanti[["DEN_REG"]]

In [ ]:
ax = regioniNONConfinanti.plot(figsize = (12, 8), edgecolor = "k", facecolor = "none")  
ctx.add_basemap(ax)